# Goal: receive sensor data from the application PhonePi

### STARTING

See https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html

See https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks

See https://www.w3schools.com/python

The application PhonePi See: https://play.google.com/store/apps/details?id=com.phonepi

# HANDS ON  

* title
    * a
    * b
     * c

pip3 install flask
pip3 install flask_sockets
pip install paho-mqtt

RUN
python PhonePiPython3.py

$e^{i\pi} + 1 = 0$

# Zen of python

In [ ]:
import this

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
costs = [1.0,4.5,9.5,16.0]
#plt.ylabel('some numbers')
#plt.plot(costs)
plt.plot([1.0, 2, 3, 4.0], costs)
plt.show()

## Sender

In [ ]:
import time
import paho.mqtt.client as paho
broker="192.168.1.6"
client= paho.Client("mqttpythonsender") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
print("connecting to broker ",broker)
client.connect(broker)                       #connect
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","2")         #publish
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","4")         #publish
time.sleep(1)
print("publishing ")
client.publish("unibo/livestream","9")         #publish
time.sleep(1)
client.disconnect()                        #disconnect
print("bye" )

## mqttReceiver.py

In [ ]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration = 20
data     = []
counter  = 0
firstVal = 0
rot      = 0

def on_message(client, userdata, message) :   #define callback
    global counter, rot, data, firstVal   
    if rot > 90  : 
        client.disconnect()             #disconnect
        return    
    msg      = str( message.payload.decode("utf-8")  )
    msgitems = msg.split(",")
    value    = msgitems[5]
    v       =  float( msgitems[5] )  
    #print( v )
    if counter <= 2 :
        firstVal  = v
        rot       = 0
        print("first",firstVal, "rot", rot, "count",counter)
    rot  = abs( v -  firstVal )
    print("rot:", rot, "count",counter )
    if rot < 90  :
        counter = counter + 1
        data.append( rot )  
        if rot > 85  :
        	print("WARNING ... rot=", rot )
    else :
        data.append( rot ) 
        print("first",firstVal, "rot", rot, "count",counter)
        plt.plot(data)
        plt.show()   
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop 

   

## mqttPlotQakEvents.py

In [ ]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration = 20
data     = []
counter  = 0 
def on_message(client, userdata, message) :   #define callback
	global counter,  data
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	#print("evMsg=", evMsg )
	msgitems = evMsg.split(",")
	v        =  float( msgitems[5] ) #5 => X (AZIMUTH for rotation)
	#print("v=", v )
	if counter	< 20  :
		counter = counter + 1
		data.append( v )
	else  :
		plt.plot(data)
		plt.show()
		data = []
		counter = 0       
 	
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        


### gyro.py

In [3]:
import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration  = 15
x         = []
y         = []
z         = []
r         = []
counter   = 0 
goon      = True
dt        = 0
startTime = 0
angle     = 0.0

def on_message(client, userdata, message) :   #define callback
	global counter,  goon, x, y, z, r, startTime, angle
	now     = time.time() 
	elapsed = now-startTime
	#print( "now=" , now, "elapsed=",  elapsed)
	startTime = now
	counter = counter + 1
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	#print("evMsg=", evMsg )
	msgitems = evMsg.split(",")
	x.append( float( msgitems[5] ) )
	y.append( float( msgitems[6] ) )
	vz = float( msgitems[7].split(')')[0] )
	z.append( vz )
	
	da    = abs(vz) * elapsed
	angle = angle + da
	r.append( angle )
	print("vz=", ("%.3f" % vz), "angle=", ("%.3f" % angle), "counter=", counter)
	#print("elapsed=", ("%.2f" % elapsed), "da=", ("%.2f" % da) ) 	
	if angle > 90.0 and goon :
		plt.plot(x, color='red')
		plt.plot(y, color='green')
		plt.plot(z, color='blue')
		plt.plot(r, color='black')
		plt.show()
		goon = False
		client.disconnect()
     
plt.style.use("classic")
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
startTime     = time.time() 
#print( "startTime=" , ti
print( "startTime=" , startTime )
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        
     


connected to broker  localhost
subscribing to unibo/qak/events
startTime= 1566296789.9229608
vz= 0.198 angle= 0.425 counter= 1
vz= -5.580 angle= 1.034 counter= 2
vz= -18.901 angle= 3.113 counter= 3
vz= -26.458 angle= 5.838 counter= 4
vz= -43.954 angle= 10.409 counter= 5
vz= -53.198 angle= 15.889 counter= 6
vz= -33.053 angle= 19.426 counter= 7
vz= -32.779 angle= 22.999 counter= 8
vz= -27.756 angle= 25.857 counter= 9
vz= -30.214 angle= 28.939 counter= 10
vz= -30.771 angle= 32.232 counter= 11
vz= -44.573 angle= 36.867 counter= 12
vz= -37.305 angle= 40.821 counter= 13
vz= -29.901 angle= 44.589 counter= 14
vz= -27.985 angle= 46.940 counter= 15
vz= -37.847 angle= 50.914 counter= 16
vz= -42.176 angle= 55.469 counter= 17
vz= -22.908 angle= 57.828 counter= 18
vz= -37.924 angle= 61.810 counter= 19
vz= -47.511 angle= 66.799 counter= 20
bye
